<a href="https://colab.research.google.com/github/jmagdeska/mlai/blob/master/%5BMLAI%5D_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environment**

In [0]:
!pip3 install 'keras'

**Libraries**

In [0]:
import keras